In [29]:
#@title Librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp
from scipy.integrate import simpson, romberg, trapezoid
from scipy.special import roots_legendre

# Actividad 07: Integración

---
### Profesor: Juan Marcos Marín
### Nombre: ______
*Métodos computacionales 2024-II*

---

#1
* Implemente una función para el **método de integración de Romberg** definiendo un límite de tolerancia de 1e-8 y/o un máximo de iteraciones de 10.

* Encuentre la integral para

$$\int_0^{\pi/4} dx\, e^{3x}\cdot \sin(x)$$

* Imprima su resultado y compare los valores dados por `scipy.integrate.romberg`

* Finalmente, encuentre el valor del error, hallando el valor exacto usando `sympy`



In [30]:
#a)
def romberg_int(f, a, b, eps, max_iter):
  R = np.zeros((max_iter, max_iter))

  n = 1  #Primer valor de n
  h = b - a
  R[0, 0] = (h / 2) * (f(a) + f(b))

  for i in range(1, max_iter):
    n *= 2
    h /= 2
    suma = sum(f(a + (k - 0.5) * h) for k in range(1, n, 2))
    R[i, 0] = 0.5 * R[i-1, 0] + h * suma

    for j in range(1, i + 1):
      R[i, j] = R[i, j-1] + (R[i, j-1] - R[i-1, j-1]) / (4**j- 1)

    if abs(R[i, i] - R[i-1, i-1]) < eps:
      return R[i, i], R[:i+1, :i+1]

  return R[i, i], R[:i+1, :i+1]


#b)
f1 = lambda x: np.exp(x*3) * np.sin(x)
romberg_int(f1, 0, np.pi/4, 1e-8, 10)

(1.576039519844102,
 array([[2.929727  , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.60293886, 1.16067614, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.23092197, 1.10691635, 1.10333236, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.22551409, 1.22371146, 1.2314978 , 1.23353218, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.31616826, 1.34638632, 1.35456464, 1.35651808, 1.35700038,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [1.40803957, 1.43866333, 1.44481514, 1.44624768, 1.44659956,
         1.44668715, 0.        , 0.        , 0.        , 0.        ],
        [1.47709534, 1.50011394, 1.50421064, 1.50515343, 1.50538443,
         1.50544189, 1.50545624, 0.        , 0.        , 0.        ],
       

In [31]:
#c)
romberg(f1, 0, np.pi/4, show=True)

Romberg integration of <function vectorize1.<locals>.vfunc at 0x7b8de774a340> from [0, 0.7853981633974483]

 Steps  StepSize   Results
     1  0.785398  2.929727 
     2  0.392699  1.952999  1.627423 
     4  0.196350  1.684140  1.594520  1.592326 
     8  0.098175  1.615224  1.592253  1.592101  1.592098 
    16  0.049087  1.597887  1.592107  1.592098  1.592098  1.592098 
    32  0.024544  1.593545  1.592098  1.592098  1.592098  1.592098  1.592098 

The final result is 1.592097707858613 after 33 function evaluations.


<ipython-input-31-ebd20f5d234d>:2: DeprecationWarning: `scipy.integrate.romberg` is deprecated as of SciPy 1.12.0and will be removed in SciPy 1.15.0. Please use`scipy.integrate.quad` instead.
  romberg(f1, 0, np.pi/4, show=True)


1.592097707858613

In [32]:
#d)
x = sp.symbols('x')
f1_ = sp.exp(x*3) * sp.sin(x)
sp.integrate(f1_, (x, 0, sp.pi/4))

1/10 + sqrt(2)*exp(3*pi/4)/10

#2

* Usando los *métodos trapezoidal compuesto*, *simpson 1/3* y de *medio punto* encuentre la siguiente integral,

$$\int_e^{1+e} dx\, \frac{1}{x\ln x}$$

* Luego, haga un estudio de la convergencia en términos del valor de $h$ o de los sub-intervalos de la función. ¿Cuál es mejor?


In [33]:
#Funciones
def simpson13(f,a,b,n):
  h = (b-a)/n
  integral = f(a) + f(b)

  #Pares
  for i in range(2, n, 2):
    xi = a + i*h
    integral += 2*f(xi)

  #Impares
  for i in range (1, n, 2):
    xi = a + i*h
    integral += 4*f(xi)
  return h/3 * integral

def trapecio(f,a,b,n):
  h = (b-a)/n
  r = 0
  for i in range(1,n):
    r += f(a+i*h)
  return (h/2)*(f(a) + 2*r + f(b))

def punto_medio(f,a,b,n):
  h = (b-a)/n
  r = 0
  for i in range(0, n):
    x_i = a + i * h
    r += f(x_i + h/2)
  return h * r


In [34]:
f2 = lambda x: 1/(x*np.log(x))

print(f"El valor con simpson 1/3 es {simpson13(f2, np.e, np.e + 1, 10)}")
print(f"El valor con trapecio es {trapecio(f2, np.e, np.e + 1, 10)}")
print(f"El valor con punto medio es {punto_medio(f2, np.e, np.e + 1, 10)}")

x = sp.symbols('x')
f2_ = 1/(x*sp.log(x))
valor_real = sp.integrate(f2_,(x,np.e, np.e + 1))

print(f"El valor real es {valor_real}")

El valor con simpson 1/3 es 0.2725141807832366
El valor con trapecio es 0.2726585180886651
El valor con punto medio es 0.27244159005549795
El valor real es 0.272513880502583


In [35]:
for i in range(1,200):
  a = simpson13(f2, np.e, np.e + 1, i)
  if np.abs(valor_real-a)<1e-5:
    print(f"Para el método de simpson es mejor tomar {i} intervalos")
    break

for i in range(1,200):
  a = trapecio(f2, np.e, np.e + 1, i)
  if np.abs(valor_real-a)<1e-5:
    print(f"Para el método de trapecio es mejor tomar {i} intervalos")
    break

Para el método de simpson es mejor tomar 6 intervalos
Para el método de trapecio es mejor tomar 39 intervalos


#3
Usando la siguiente función:



```python
def gauss_quad_standard(func, n):
    """
    Calcula la integral de una función en el intervalo [-1, 1]
    utilizando cuadratura gaussiana.

    Parameters:
    - func: La función a integrar.
    - n: Número de puntos para la cuadratura (grado del polinomio de Legendre).

    Returns:
    - Aproximación de la integral.
    """
    # Obtener raíces y pesos del polinomio de Legendre
    x, w = roots_legendre(n)

    # Evaluar la suma ponderada
    integral = np.sum(w * func(x))
    return integral
```

Modifique la función `gauss_quad_standard` de forma tal que no este restringida para $[-1,1]$ sino para cualquier intervalo $[a,b]$. Luego, encuentre la integral del *punto 2*.





In [36]:
#Código modificado

def gauss_quad_standard(func, a, b, n):
    """
    Calcula la integral de una función
    utilizando cuadratura gaussiana.

    Parameters:
    - func: La función a integrar.
    - n: Número de puntos para la cuadratura (grado del polinomio de Legendre).

    Returns:
    - Aproximación de la integral.
    """
    import sympy as sp
    x = sp.Symbol('x')

    # Obtener raíces y pesos del polinomio de Legendre
    xi, w = roots_legendre(n)

    func_ = func(np.abs((b-a)/2*x + (a+b)/2))
    f = sp.lambdify(x, func_)

    # Evaluar la suma ponderada
    integral = (b-a)/2 * np.sum(w * f(xi))
    return integral

#Funcion punto 2
f3 = lambda x: 1/(x*sp.log(x))
gauss_quad_standard(f3, np.e, np.e+1, 8)


0.2725138805025821

#4

Encuentra todas las raices para los polinomios de grado 3 y 4 de **Legendre** usando el Método de la Secante y Newton-Raphson.



```python
import sympy as sp
x = sp.Symbol('x')

# Polinomio de Legendre de grado n
Pn = sp.legendre(n, x)

```

y calcule los pesos $w_i$ de la cuadratura mediante la fórmula:
   $$
   w_i = \frac{2}{(1 - x_i^2) \left[P_n'(x_i)\right]^2},
   $$
   donde $P_n'(x)$ es la derivada del polinomio de Legendre $P_n(x)$.


In [37]:
import sympy as sp
from scipy.optimize import newton
import pandas as pd
x = sp.Symbol('x')

# Polinomio de Legendre de grado 3
p3 = sp.legendre(3, x)
dp3 = sp.diff(p3, x)

p3_func = sp.lambdify(x, p3, "numpy")
dp3_func = sp.lambdify(x, dp3, "numpy")

#Raices del polinomio método de Newton
raiz1 = newton(p3_func,-1,dp3_func)
raiz2 = newton(p3_func,0.2,dp3_func)
raiz3 = newton(p3_func,1,dp3_func)

lista_raices3 = [raiz1, raiz2, raiz3]
lista_pesos3 = []

for i in lista_raices3:
  wi = 2/((1 - i**2)*(dp3_func(i))**2)
  lista_pesos3.append(wi)


# Polinomio de Legendre de grado 4
p4 = sp.legendre(4, x)
dp4 = sp.diff(p4, x)

p4_func = sp.lambdify(x, p4, "numpy")
dp4_func = sp.lambdify(x, dp4, "numpy")

#Raices del polinomio método de la secante
raiz1_ = newton(p4_func,-1)
raiz2_= newton(p4_func,-0.2)
raiz3_ = newton(p4_func,0.2)
raiz4_ = newton(p4_func,1)

lista_raices4 = [raiz1_, raiz2_, raiz3_, raiz4_]
lista_pesos4 = []

for i in lista_raices4:
  wi = 2/((1 - i**2)*(dp3_func(i))**2)
  lista_pesos4.append(wi)

datos3 = {"Raices": lista_raices3,
          "Pesos": lista_pesos3}

df3 = pd.DataFrame(datos3)

datos4 = {"Raices": lista_raices4,
          "Pesos": lista_pesos4}

df4 = pd.DataFrame(datos4)

In [38]:
print("Para el polinomio de grado 3")
df3

Para el polinomio de grado 3


,Raices,Pesos
0,-0.774597,0.555556
1,0.000000,0.888889
2,0.774597,0.555556


In [39]:
print("Para el polinomio de grado 4")
df4

Para el polinomio de grado 4


,Raices,Pesos
0,-0.861136,0.469088
1,-0.339981,5.642023
2,0.339981,5.642023
3,0.861136,0.469088


In [40]:
#Raices del polinomio método de Secante
raiz1 = newton(p3_func,-1)
raiz2 = newton(p3_func,0.2)
raiz3 = newton(p3_func,1)

lista_raices3 = [raiz1, raiz2, raiz3]
lista_pesos3 = []

for i in lista_raices3:
  wi = 2/((1 - i**2)*(dp3_func(i))**2)
  lista_pesos3.append(wi)


# Polinomio de Legendre de grado 4
p4 = sp.legendre(4, x)
dp4 = sp.diff(p4, x)

p4_func = sp.lambdify(x, p4, "numpy")
dp4_func = sp.lambdify(x, dp4, "numpy")

#Raices del polinomio método de la secante
raiz1_ = newton(p4_func,-1)
raiz2_= newton(p4_func,-0.2)
raiz3_ = newton(p4_func,0.2)
raiz4_ = newton(p4_func,1)

lista_raices4 = [raiz1_, raiz2_, raiz3_, raiz4_]
lista_pesos4 = []

for i in lista_raices4:
  wi = 2/((1 - i**2)*(dp3_func(i))**2)
  lista_pesos4.append(wi)

datos3 = {"Raices": lista_raices3,
          "Pesos": lista_pesos3}

df3 = pd.DataFrame(datos3)

datos4 = {"Raices": lista_raices4,
          "Pesos": lista_pesos4}

df4 = pd.DataFrame(datos4)

In [41]:
print("Para el polinomio de grado 3")
df3

Para el polinomio de grado 3


,Raices,Pesos
0,-7.745967e-01,0.555556
1,-4.393205e-22,0.888889
2,7.745967e-01,0.555556


In [42]:
print("Para el polinomio de grado 4")
df4

Para el polinomio de grado 4


,Raices,Pesos
0,-0.861136,0.469088
1,-0.339981,5.642023
2,0.339981,5.642023
3,0.861136,0.469088
